In [ ]:
import os
import json

In [ ]:
def hypodd_ct(
    node_i: int,
    config_json: "json",
    ct_file:str,
    event: str,
    station: str,
    catalog_txt:str,
    data_path: str ,
):
    import json
    import os
    from datetime import datetime

    with open(config_json, "r") as fp:
        config = json.load(fp)
    hypodd_path = os.path.join(data_path, "hypodd")

    ct_inp = """* RELOC.INP:
*--- input file selection
* cross correlation diff times:

*
*catalog P diff times:
dt.ct
*
* event file:
event.sel
*
* station file:
stations.dat
*
*--- output file selection
* original locations:
hypodd.loc
* relocations:
hypodd.reloc
* station information:
hypodd.sta
* residual information:
hypodd.res
* source paramater information:
hypodd.src
*
*--- data type selection: 
* IDAT:  0 = synthetics; 1= cross corr; 2= catalog; 3= cross & cat 
* IPHA: 1= P; 2= S; 3= P&S
* DIST:max dist [km] between cluster centroid and station 
* IDAT   IPHA   DIST
    2     3     120
*
*--- event clustering:
* OBSCC:    min # of obs/pair for crosstime data (0= no clustering)
* OBSCT:    min # of obs/pair for network data (0= no clustering)
* OBSCC  OBSCT    
     0     8        
*
*--- solution control:
* ISTART:  	1 = from single source; 2 = from network sources
* ISOLV:	1 = SVD, 2=lsqr
* NSET:      	number of sets of iteration with specifications following
*  ISTART  ISOLV  NSET
    2        2      4
*
*--- data weighting and re-weighting: 
* NITER: 		last iteration to used the following weights
* WTCCP, WTCCS:		weight cross P, S 
* WTCTP, WTCTS:		weight catalog P, S 
* WRCC, WRCT:		residual threshold in sec for cross, catalog data 
* WDCC, WDCT:  		max dist [km] between cross, catalog linked pairs
* DAMP:    		damping (for lsqr only) 
*       ---  CROSS DATA ----- ----CATALOG DATA ----
* NITER WTCCP WTCCS WRCC WDCC WTCTP WTCTS WRCT WDCT DAMP
   4     -9     -9   -9    -9   1     1      8   -9  70 
   4     -9     -9   -9    -9   1     1      6    4  70 
   4     -9     -9   -9    -9   1    0.8     4    2  70 
   4     -9     -9   -9    -9   1    0.8     3    2  70 
*
*--- 1D model:
* NLAY:		number of model layers  
* RATIO:	vp/vs ratio 
* TOP:		depths of top of layer (km) 
* VEL: 		layer velocities (km/s)
* NLAY  RATIO 
   12     1.82
* TOP 
0.0 1.0 3.0 5.0 7.0 9.0 11.0 13.0 17.0 21.0 31.00 31.10
* VEL
5.30 5.65 5.93 6.20 6.20 6.20 6.20 6.20 6.20 6.20 7.50 8.11
*
*--- event selection:
* CID: 	cluster to be relocated (0 = all)
* ID:	cuspids of event to be relocated (8 per line)
* CID    
    0      
* ID
"""

    with open(os.path.join(hypodd_path, "ct.inp"), "w") as fp:
        fp.writelines(ct_inp)

    def copy_file(fp_from, fp_to):
        with open(fp_from, "r") as fp:
            lines = fp.readlines()
        with open(fp_to, "w") as fp:
            fp.writelines(lines)

    copy_file(ct_file, os.path.join(hypodd_path, "dt.ct"))
    copy_file(event, os.path.join(hypodd_path, "event.sel"))
    copy_file(station, os.path.join(hypodd_path, "stations.dat"))

    # os.system(f"cat {ct_file}")
    # os.system(f"cat {event}")
    # os.system(f"cat {station}")

    HYPODD_CMD = f"cd {hypodd_path} && /opt/hypodd/hypoDD ct.inp"
    print(HYPODD_CMD)
    if os.system(HYPODD_CMD) != 0:
        raise ("{HYPODD_CMD}" + " failed!")

    copy_file(os.path.join(hypodd_path, "hypodd.reloc"), catalog_txt)

    # try:
    #     from minio import Minio

    #     minioClient = Minio(s3_url, access_key="minio", secret_key="minio123", secure=secure)
    #     minioClient.fput_object(
    #         bucket_name,
    #         f"{config['region']}/hypodd/hypodd_ct_{node_i:03d}.reloc",
    #         catalog_txt,
    #     )
    # except Exception as err:
    #     print(f"ERROR: can not access minio service! \n{err}")
    #     pass

In [ ]:
with open('num_parallel.txt', 'r') as f:
    nodes = [int(x) for x in f.read().strip('[]').split(',')]

In [ ]:

hypodd_ct(
    0,
    config_json="config/config.json",
    ct_file="hypodd/dt.ct",
    event="hypodd/event.sel",
    station="hypodd/stations.dat",
    catalog_txt="hypodd_ct_catalog.txt",
    data_path="./" ,
)

In [8]:
# Kubeflow Pipelines UI用のメタデータ出力
if os.environ.get('ELYRA_RUNTIME_ENV') == 'kfp':
    # For information about Elyra environment variables refer to
    # https://elyra.readthedocs.io/en/stable/user_guide/best-practices-file-based-nodes.html#proprietary-environment-variables

    metadata = {
        'outputs': [
            {
                'storage': 'inline',
                'source': f'# Run HypoDD re-location Complete\n...',
                'type': 'markdown',
            }
        ]
    }

    with open('mlpipeline-ui-metadata.json', 'w') as f:
        json.dump(metadata, f)